<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/main/Basics_of_CUDA_with_numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numba

In [2]:
import numpy as np
from numba import cuda

CUDA Kernel is function that will be called from CPU


*   **Kernels can't return a value:** A kernel function (in GPU programming) can't
    directly return a result. Instead, it writes its results to an array that you pass to it. If the result is just one value (a scalar), you'd use a one-element array to store that value.

*   **Thread hierarchy in kernels:** When calling a kernel, you must specify how many thread blocks (groups of threads) and how many threads per block it will use. Although the kernel code is compiled once, you can call it multiple times with different numbers of blocks or threads, depending on the task.


CUDA **thread** are the single unit of computation: It represent a smallest independent task, a single thread process specific data element and many work together to bring parallelism in massive amount.

Key Points about Threads:

* Single task: Each thread runs a copy of the same kernel code but operates on different data (based on its unique thread ID).

* Parallel execution: Thousands of threads can run simultaneously on the GPU, allowing massive parallelism.

* Thread IDs: Threads are uniquely identified within their block (using threadIdx.x, threadIdx.y, threadIdx.z), and you can use this ID to determine which part of the data the thread processes.

```
import numpy as np
from numba import cuda

# Define the CUDA kernel in Numba
@cuda.jit
def add_arrays(a, b, result, size):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x  # Calculate thread's global index
    if idx < size:
        result[idx] = a[idx] + b[idx]  # Perform the addition

# Initialize input arrays
size = 100
a = np.random.randint(0, 100, size).astype(np.int32)  # Random integers array
b = np.random.randint(0, 100, size).astype(np.int32)
result = np.zeros(size, dtype=np.int32)  # Output array

# Copy data to the GPU
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_result = cuda.to_device(result)

# Define number of threads and blocks
threads_per_block = 32
blocks_per_grid = (size + (threads_per_block - 1)) // threads_per_block  # Ensure full coverage

# Launch the kernel
add_arrays[blocks_per_grid, threads_per_block](d_a, d_b, d_result, size)

# Copy the result back to the CPU
result = d_result.copy_to_host()

# Display the result
print(result)

```

1. **Threads per block (threads_per_block = 32):**
This defines how many threads you want in each block. In this case, we are assigning 32 threads to each block.
 The maximum number of threads per block typically ranges from 32 to 1024, depending on the GPU.
Choosing 32 threads is a common choice because it's the size of a "warp" on most GPUs, which is a group of threads that execute together in lockstep (for efficiency).
2. **Blocks per grid (blocks_per_grid = (size + (threads_per_block - 1)) // threads_per_block):**
**This calculates the number of blocks needed to ensure that every element of the array is processed by a thread.**

Breakdown of the formula:

**size** : This is the total number of elements in the arrays a, b, and result that need to be processed (in this case, 100 elements).

**threads_per_block - 1** : This is added to the array size to handle cases where size isn’t a perfect multiple of threads_per_block. Adding (threads_per_block - 1) ensures that any remaining elements in the array (if size isn't divisible by threads_per_block) are still covered by an additional block.

***// threads_per_block: This is integer division. It determines how many full blocks are needed. For example:***

If size = 100 and threads_per_block = 32, then:$$
\text{blocks_per_grid} = \frac{100 + 31}{32} = \frac{131}{32} = 4 \text{ blocks}
$$ This ensures there are enough blocks to cover all elements. In this example, 4 blocks of 32 threads will provide coverage for 128 total threads, which is more than enough to process all 100 elements.
Why This Formula Is Used:
CUDA organizes threads into blocks. If the size of the data (size = 100) is not divisible by the number of threads per block (threads_per_block = 32), there will be leftover elements that won’t get processed unless an additional block is added. The formula ensures that even if there are leftover elements, they will be covered by an extra block.

This approach guarantees full coverage of the data, even when the number of elements doesn't perfectly fit into the blocks.

**In Summary:**

**threads_per_block** specifies how many threads each block has (32 here).

**blocks_per_grid** calculates how many blocks are needed to make sure every element in the array gets processed, ensuring that no data is left out.


In [3]:
@cuda.jit
def add_arrays(a, b, result, size):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x  # Calculate thread's global index
    if idx < size:
        result[idx] = a[idx] + b[idx]  # Perform the addition

**Explanation:**

A thread can hold **multiple vairbales** and **manupulate** them.

`cuda.threadIdx.x:` Index of the thread within its block.

`cuda.blockIdx.x:` Index of the block within the grid.

`cuda.blockDim.x:` Total number of threads per block.

**Purpose:**

This expression calculates the global index (idx) for the thread, allowing it to uniquely identify which data element to process by combining the thread's index within its block and the total number of threads in all previous blocks.

**Example:**

For Thread 0 in Block 1 with `blockDim.x = 32`:

`idx = 0 + (1 * 32) = 32` This means it processes the 33rd element of the data.

In [6]:
# Initialize input arrays
size = 100
a = np.random.randint(0, 100, size).astype(np.int32)
b = np.random.randint(0, 100, size).astype(np.int32)
result = np.zeros(size, dtype=np.int32)  # Output array

# Copy data to the GPU
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_result = cuda.to_device(result)

# Define number of threads and blocks
threads_per_block = 32
blocks_per_grid = (size + (threads_per_block - 1)) // threads_per_block  # Ensure full coverage

print(f"Number of threads per block: {blocks_per_grid}")
print(f"Number of blocks needed to fit all the element of data: {threads_per_block}")
print(f"Total number of threads: {blocks_per_grid * threads_per_block}")

Number of threads per block: 4
Number of blocks needed to fit all the element of data: 32
Total number of threads: 128


In [9]:
# Launch the kernel
add_arrays[blocks_per_grid, threads_per_block](d_a, d_b, d_result, size)

# Copy the result back to the CPU
result = d_result.copy_to_host()

# Display the result
print(result)
print(f"Len of result : {len(result)}")

[ 39  74  64  75  67 148 113 146 101 166  88  69 150 115  27 109  85 100
 139  46 122 143 120  60  69 137  68 109  81  65  96  49  64  91 179 107
  43  81 125 126  86 165 100  96  84  92  52 166 159  93   4  75  28  99
 113  62  75  43 133 113 127 128  96  65  63  55  93  50  53 122  69  16
  73  83 125  28  98 147 104 138 100 177 154  80 157  60  52 145  25  74
 136 103 112  14  85 110 170  20 120 148]
Len of result : 100


In [7]:
%%timeit
add_arrays[blocks_per_grid, threads_per_block](d_a, d_b, d_result, size)

67.9 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
